Load Julia packages (libraries) needed  for the snippets in chapter 0

In [1]:
using DynamicHMCModels

CmdStan uses a tmp directory to store the output of cmdstan

In [2]:
ProjDir = rel_path_d("..", "scripts", "05")
cd(ProjDir)

Read the milk data

In [3]:
wd = CSV.read(rel_path("..", "data", "milk.csv"), delim=';')
df = convert(DataFrame, wd);
dcc = filter(row -> !(row[:neocortex_perc] == "NA"), df)
dcc[:kcal_per_g] = convert(Vector{Float64}, dcc[:kcal_per_g])
dcc[:log_mass] = log.(convert(Vector{Float64}, dcc[:mass]))

┌ Warning: `getindex(df::DataFrame, col_ind::ColumnIndex)` is deprecated, use `df[!, col_ind]` instead.
│   caller = top-level scope at string:4
└ @ Core string:4
┌ Warning: `setindex!(df::DataFrame, v::AbstractVector, col_ind::ColumnIndex)` is deprecated, use `begin
│     df[!, col_ind] = v
│     df
│ end` instead.
│   caller = top-level scope at string:4
└ @ Core string:4
┌ Warning: `getindex(df::DataFrame, col_ind::ColumnIndex)` is deprecated, use `df[!, col_ind]` instead.
│   caller = top-level scope at string:5
└ @ Core string:5
┌ Warning: `setindex!(df::DataFrame, v::AbstractVector, col_ind::ColumnIndex)` is deprecated, use `begin
│     df[!, col_ind] = v
│     df
│ end` instead.
│   caller = top-level scope at string:5
└ @ Core string:5


17-element Array{Float64,1}:
  0.6678293725756554
  1.6582280766035324
  1.6808279085207734
  0.9202827531436925
 -0.3856624808119846
 -2.120263536200091 
 -0.7550225842780328
 -1.1394342831883648
  0.4382549309311553
  1.1755733298042381
  2.509599262378372 
  1.6808279085207734
  3.5689691574413787
  4.374876130645041 
  3.70721041079866  
  3.4998353515591547
  4.006423680849631 

Show first 5 rows

In [4]:
first(dcc[[3, 7, 9]], 5)

┌ Warning: `getindex(df::DataFrame, col_inds::Union{AbstractVector, Regex, Not})` is deprecated, use `df[:, col_inds]` instead.
│   caller = top-level scope at string:1
└ @ Core string:1


,kcal_per_g,mass,log_mass
,Float64,Float64,Float64
1,0.49,1.95,0.667829
2,0.47,5.25,1.65823
3,0.56,5.37,1.68083
4,0.89,2.51,0.920283
5,0.92,0.68,-0.385662


Define the model struct

In [5]:
struct m_5_6{TY <: AbstractVector, TX <: AbstractMatrix}
    "Observations."
    y::TY
    "Covariates"
    X::TX
end

Make the type callable with the parameters *as a single argument*.

In [6]:
function (problem::m_5_6)(θ)
    @unpack y, X, = problem   # extract the data
    @unpack β, σ = θ            # works on the named tuple too
    ll = 0.0
    ll += logpdf(Normal(0, 100), X[1]) # a = X[1]
    ll += logpdf(Normal(0, 1), X[2]) # b1 = X[2]
    ll += logpdf(TDist(1.0), σ)
    ll += loglikelihood(Normal(0, σ), y .- X*β)
    ll
end

Instantiate the model with data and inits.

In [7]:
N = size(dcc, 1)
X = hcat(ones(N), dcc[:log_mass]);
y = dcc[:kcal_per_g]
p = m_5_6(y, X);
p((β = [1.0, 2.0], σ = 1.0))

┌ Warning: `getindex(df::DataFrame, col_ind::ColumnIndex)` is deprecated, use `df[!, col_ind]` instead.
│   caller = top-level scope at string:2
└ @ Core string:2
┌ Warning: `getindex(df::DataFrame, col_ind::ColumnIndex)` is deprecated, use `df[!, col_ind]` instead.
│   caller = top-level scope at string:3
└ @ Core string:3


-242.8761844035513

Write a function to return properly dimensioned transformation.

In [8]:
problem_transformation(p::m_5_6) =
    as((β = as(Array, size(p.X, 2)), σ = asℝ₊))

problem_transformation (generic function with 1 method)

Wrap the problem with a transformation, then use Flux for the gradient.

In [9]:
P = TransformedLogDensity(problem_transformation(p), p)
∇P = LogDensityRejectErrors(ADgradient(:ForwardDiff, P));

Tune and sample.

In [10]:
chain, NUTS_tuned = NUTS_init_tune_mcmc(∇P, 1000);

MCMC, adapting ϵ (75 steps)
0.00015 s/step ...done
MCMC, adapting ϵ (25 steps)
0.00023 s/step ...done
MCMC, adapting ϵ (50 steps)
0.00012 s/step ...done
MCMC, adapting ϵ (100 steps)
8.5e-5 s/step ...done
MCMC, adapting ϵ (200 steps)
6.5e-5 s/step ...done
MCMC, adapting ϵ (400 steps)
6.1e-5 s/step ...done
MCMC, adapting ϵ (50 steps)
9.0e-5 s/step ...done
MCMC (1000 steps)
0.0001 s/step ...done


We use the transformation to obtain the posterior from the chain.

In [11]:
posterior = TransformVariables.transform.(Ref(problem_transformation(p)), get_position.(chain));
posterior[1:5]

5-element Array{NamedTuple{(:β, :σ),Tuple{Array{Float64,1},Float64}},1}:
 (β = [0.6126020723418151, -0.012610725335387356], σ = 0.16754342414431214)
 (β = [0.8007898561249145, -0.05323138512639664], σ = 0.17051470864047646) 
 (β = [0.7410651436062766, -0.05058922853928243], σ = 0.1469105181004563)  
 (β = [0.6009204543459268, 0.0039654462869644636], σ = 0.1484827748817822) 
 (β = [0.7465029386299306, -0.04264231209378815], σ = 0.18806099850362162) 

Extract the parameter posterior means: `β`,

In [12]:
posterior_β = mean(first, posterior)

2-element Array{Float64,1}:
  0.7041385209035523 
 -0.03116472658921996

then `σ`:

In [13]:
posterior_σ = mean(last, posterior)

0.1848558162433723

Effective sample sizes (of untransformed draws)

In [14]:
ess = mapslices(effective_sample_size,
                get_position_matrix(chain); dims = 1)

1×3 Array{Float64,2}:
 485.828  812.149  470.359

NUTS-specific statistics

In [15]:
NUTS_statistics(chain)

Hamiltonian Monte Carlo sample of length 1000
  acceptance rate mean: 0.9, min/25%/median/75%/max: 0.05 0.86 0.95 0.99 1.0
  termination: AdjacentTurn => 26% DoubledTurn => 74%
  depth: 1 => 5% 2 => 55% 3 => 35% 4 => 4% 5 => 0%


cmdstan result

In [16]:
cmdstan_result = "
Iterations = 1:1000
Thinning interval = 1
Chains = 1,2,3,4
Samples per chain = 1000

Empirical Posterior Estimates:
          Mean         SD        Naive SE       MCSE      ESS
    a  0.70472876 0.057040655 0.00090189195 0.0011398893 1000
   bm -0.03150330 0.023642759 0.00037382484 0.0004712342 1000
sigma  0.18378372 0.039212805 0.00062000888 0.0011395979 1000

Quantiles:
          2.5%       25.0%       50.0%        75.0%       97.5%
    a  0.59112968  0.66848775  0.70444950  0.741410500 0.81915225
   bm -0.07729257 -0.04708425 -0.03104865 -0.015942925 0.01424901
sigma  0.12638780  0.15605950  0.17800600  0.204319250 0.27993590
";

Extract the parameter posterior means: `[β, σ]`,

In [17]:
[posterior_β, posterior_σ]

2-element Array{Any,1}:
  [0.7041385209035523, -0.03116472658921996]
 0.1848558162433723                         

End of `05/5.6d.jl`

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*